In [0]:
import pdb
import h5py

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.losses import *
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt


import os
import pdb
import glob
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.losses import *

Using TensorFlow backend.


In [0]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Directories
train_root = '/content/drive/My Drive/Colab Notebooks/data/train'
valid_root = '/content/drive/My Drive/Colab Notebooks/data/test'
test_root = '/content/drive/My Drive/Colab Notebooks/data/test'
gt_root = '/content/drive/My Drive/Colab Notebooks/data/test/label'
test_code_root = '/content/drive/My Drive/Colab Notebooks/test_code'
output_root = '/content/drive/My Drive/Colab Notebooks/output/outputs'
train_ckpt_path = '/content/drive/My Drive/Colab Notebooks/output/unet.hdf5'


# Variables
resume = False
input_size = (240,240,1)


# Hyperparameters
epoch = 20
lr = 0.0001
train_batch_size = 33
test_batch_size = 88
input_size = (240,240,1)

In [0]:

# label = cv2.imread(os.path.join(test_root,'label/000.jpg')) 
# print(label.shape)
# label = np.expand_dims(label, axis=2)*255
# print(label.shape)
# zeros = np.zeros(label.shape)
# label = np.concatenate((zeros,zeros,label), axis=2)  
# print(label.shape)
# gt_img = cv2.imread(os.path.join(gt_root, '000.png'))
# print(gt_img)
# gt_img = gt_img+label
# gt_img = (gt_img/(gt_img.max()+1e-6))*255
# cv2.imwrite(gt_root, gt_img)


In [0]:

def bce_loss(y_true, y_pred):
    ''' Binary Cross Entropy

    Args:
        y_true (np.array): prediction
        y_pred (np.array): ground-truth
    '''
    return binary_crossentropy(y_true, y_pred)
  
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_numpy(y_true, y_pred):
    smooth = 1.
    y_true_f = np.ndarray.flatten(y_true)
    y_pred_f = np.ndarray.flatten(y_pred)
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def iou_score(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (1. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_coef_log_loss(y_true, y_pred):
    return -K.log(dice_coef(y_true, y_pred))

def iou_score_loss(y_true, y_pred):
    return -iou_score(y_true, y_pred)

def plt_img(img=None, img_path=None, save_path=None):
    ''' Plot Image using given image or image_path

    Args:
        img         (np.array)  : image to plot
        img_path    (str)       : image path to plot
        save_path   (str)       : save directory for plotted image
    '''
    if isinstance(save_path, type(None)):
        save_path = './plt_img.jpg'

    if not(isinstance(img, type(None))):
        #cv2.imwrite(save_path,img)
        plt.rcParams['figure.figsize'] = [10, 30]
        plt.imshow(img, clim=(0, 255))
        #plt.imshow(np.rot90(img, -1), clim=(0, 255))

    if not(isinstance(img_path, type(None))):
        img = cv2.imread(img_path)
        #cv2.imwrite(save_path, img)
        plt.imshow(img, clim=(0, 255))
        #plt.imshow(np.rot90(img, -1), clim=(0, 255))




In [ ]:
def adjust_data(image, label):
    ''' Adjust images and labels for network inputs

    Args:
        image   (np.array):   Augmented images
        label   (np.array):   Augmented labels
    '''
    image = image[0] / 255
    label = label[0]/255
    label[label>0.5] =1
    label[label<=0.5]=0
    return image, label


def dataset(mode, data_root, batch_size,
            image_color_mode = "grayscale",
            label_color_mode = "grayscale",
            target_size = (240,240)):

    ''' Prepare dataset ( pre-processing + augmentation(optional) )

    Args:
        args (argparse):          Arguments parsered in command-lind
        image_color_mode (str):   Image color Mode Flag
        label_color_mode (str):   Label color Mode Flag
        target_size (tuple):      Target Size
    '''
    if mode == 'train':
        shuffle=True
        # Data Augmentation (image)
        image_datagen = ImageDataGenerator(rotation_range=15,
                                           horizontal_flip=True,
                                           vertical_flip=False,
                                           width_shift_range=0.1,
                                           height_shift_range=0.1,
                                           shear_range=0.2,
                                           zoom_range=0.1)

        # Data Augmentation (label)
        label_datagen = ImageDataGenerator(rotation_range=15,
                                           horizontal_flip=True,
                                           vertical_flip=False,
                                           width_shift_range=0.1,
                                           height_shift_range=0.1,
                                           shear_range=0.2,
                                           zoom_range=0.1)

        print('Train Set is prepared')

    elif mode == 'valid':
        shuffle=False
        image_datagen = ImageDataGenerator()
        label_datagen = ImageDataGenerator()

        print('Validation Set is prepared')

    elif mode == 'test':
        shuffle=False
        image_datagen = ImageDataGenerator()
        label_datagen = ImageDataGenerator()

        print('Test Set is prepared')

    else:
        raise ValueError('Dataset Mode ERROR')

    # Dataloader (image)
    image_generator = image_datagen.flow_from_directory(
        data_root,
        classes=['image'],
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        shuffle=shuffle,
        seed=1)

   # Dataloader (label)
    label_generator = label_datagen.flow_from_directory(
        data_root,
        classes=['label'],
        color_mode=label_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        shuffle=shuffle,
        seed=1)

    data_generator = zip(image_generator, label_generator)
    for i, (image, label) in enumerate(data_generator):
        image, label = adjust_data(image, label)
        ##Save augmentated images and labels
        #for j in range(image.shape[0]):
        #    cv2.imwrite(str(i)+'-'+str(j)+'.jpg', image[j].squeeze()*255)
        #    cv2.imwrite(str(i)+'-'+str(j)+'_.jpg', label[j].squeeze()*255)
        yield (image, label)


def save_result(output, data_root, output_root, gt_root=None):
  
    ''' Save Prediction results overlapping on original images

    Args:
        args (argparse):    Arguments parsered in command-lind
        output (np.array):  Prediction Results by segementation model
        
    '''
    file_path = []
    file_path += glob.glob(os.path.join(data_root, 'image/*.jpg'))
    file_path = sorted(file_path)
    output = output.squeeze()
    final = None
    
    for i in range(output.shape[0]):
        if len(str(i)) == 1:
            name = '00'+str(i)
        elif len(str(i)) == 2:
            name = '0'+str(i)
        else:
            name = str(i)

        save_path = os.path.join(output_root, name+'.jpg')
        img = cv2.imread(file_path[i])
        pred = (output[i]>0.5).astype(int)
        pred = np.expand_dims(pred, axis=2)*255
        zeros = np.zeros(pred.shape)
        pred = np.concatenate((zeros,zeros,pred), axis=2)
        img = img + pred
        img = (img/(img.max()+1e-6))*255
        cv2.imwrite(save_path, img)

        # Print Example of Prediction and Ground-truth
        if (i==90 or i==63 or i==82 or i==10):
            gt_img = cv2.imread(os.path.join(gt_root, name+'.jpg'))
            img = (np.concatenate((img,gt_img),axis=1)).astype(np.uint8)
            if isinstance(final, type(None)):
                final = img
            else:
                final = np.concatenate((final, img), axis=0).astype(np.uint8)
    
    plt_img(img=final, save_path=os.path.join(test_code_root,'results.jpg'))

# Model (U-net)

In [5]:
inputs = Input(input_size)
print(inputs)

Tensor("input_1:0", shape=(?, 240, 240, 1), dtype=float32)


In [0]:
out1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
print(out1)
out1 = Conv2D(16, (3, 3), activation='relu', padding='same')(out1)
print(out1)
out2 = MaxPooling2D(pool_size=(2, 2))(out1)
print(out2)
out2 = Conv2D(32, (3, 3), activation='relu', padding='same')(out2)
print(out2)
out2 = Conv2D(32, (3, 3), activation='relu', padding='same')(out2)
print(out2)
out3 = MaxPooling2D(pool_size=(2, 2))(out2)
print(out3)
out3 = Conv2D(64, (3, 3), activation='relu', padding='same')(out3)
print(out3)
out3 = Conv2D(64, (3, 3), activation='relu', padding='same')(out3)
print(out3)
out4 = MaxPooling2D(pool_size=(2, 2))(out3)
print(out4)
out4 = Conv2D(128, (3, 3), activation='relu', padding='same')(out4)
print(out4)
out4 = Conv2D(128, (3, 3), activation='relu', padding='same')(out4)
print(out4)
out5 = MaxPooling2D(pool_size=(2, 2))(out4)
print(out5)
out5 = Conv2D(256, (3, 3), activation='relu', padding='same')(out5)
print(out5)
out5 = Conv2D(256, (3, 3), activation='relu', padding='same')(out5)
print(out5)
out4 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(out5),out4], axis=3)
print(out4)
out4 = Conv2D(128, (3, 3), activation='relu', padding='same')(out4)
print(out4)
out4 = Conv2D(128, (3, 3), activation='relu', padding='same')(out4)
print(out4)
out3 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(out4),out3], axis=3)
print(out3)
out3 = Conv2D(64, (3, 3), activation='relu', padding='same')(out3)
print(out3)
out3 = Conv2D(64, (3, 3), activation='relu', padding='same')(out3)
print(out3)
out2 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2),padding='same')(out3),out2], axis=3)
print(out2)
out2 = Conv2D(32, (3, 3), activation='relu', padding='same')(out2)
print(out2)
out2 = Conv2D(32, (3, 3), activation='relu', padding='same')(out2)
print(out2)
out1 = concatenate([Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(out2),out1], axis=3)
print(out1)
out1 = Conv2D(16, (3, 3), activation='relu', padding='same')(out1)
print(out1)
out1 = Conv2D(16, (3, 3), activation='relu', padding='same')(out1)
print(out1)
out1 = Conv2D(1, (1, 1), activation='sigmoid')(out1)
print(out1)

Instructions for updating:
Colocations handled automatically by placer.
Tensor("conv2d_1/Relu:0", shape=(?, 240, 240, 16), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(?, 240, 240, 16), dtype=float32)
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 120, 120, 16), dtype=float32)
Tensor("conv2d_3/Relu:0", shape=(?, 120, 120, 32), dtype=float32)
Tensor("conv2d_4/Relu:0", shape=(?, 120, 120, 32), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 60, 60, 32), dtype=float32)
Tensor("conv2d_5/Relu:0", shape=(?, 60, 60, 64), dtype=float32)
Tensor("conv2d_6/Relu:0", shape=(?, 60, 60, 64), dtype=float32)
Tensor("max_pooling2d_3/MaxPool:0", shape=(?, 30, 30, 64), dtype=float32)
Tensor("conv2d_7/Relu:0", shape=(?, 30, 30, 128), dtype=float32)
Tensor("conv2d_8/Relu:0", shape=(?, 30, 30, 128), dtype=float32)
Tensor("max_pooling2d_4/MaxPool:0", shape=(?, 15, 15, 128), dtype=float32)
Tensor("conv2d_9/Relu:0", shape=(?, 15, 15, 256), dtype=float32)
Tensor("conv2d_10/Relu:0", shape=(?, 15, 1

In [0]:
# Optimizier / Loss Init
model = Model(inputs=[inputs], outputs=[out1])
             
model.compile(optimizer=Adam(lr=lr), loss=dice_coef_loss, metrics=[dice_coef,bce_loss,iou_score])

# Model Train

In [0]:
# Trainset Load
trainset = dataset(mode='train', data_root=train_root, batch_size=train_batch_size)

In [0]:
# Validationset Load
validset = dataset(mode='valid', data_root=valid_root, batch_size=train_batch_size)

In [0]:
# Model Load
model_checkpoint = ModelCheckpoint(train_ckpt_path, monitor='val_loss',
                                    verbose=2, save_best_only=True)

In [0]:

# Model Train
history = model.fit_generator(trainset, steps_per_epoch=100, shuffle=True, epochs=epoch,
                    validation_data=validset, validation_steps=30,
                    callbacks=[model_checkpoint], workers=1)


Epoch 1/20
Validation Set is prepared
Train Set is prepared
Found 151 images belonging to 1 classes.
Found 565 images belonging to 1 classes.
Found 151 images belonging to 1 classes.
Found 565 images belonging to 1 classes.
 16/100 [===>..........................] - ETA: 2:40 - loss: -0.6570 - dice_coef: 0.6570 - bce_loss: 0.6850 - iou_score: 0.3285

# Model Test

In [0]:
# Data Load
testset = dataset(mode='test', data_root=test_root, batch_size=test_batch_size)

In [0]:
# Model Load
model.load_weights(train_ckpt_path)

In [22]:
# Model Test
results = model.predict_generator(testset, steps=1, verbose=1)

Test Set is prepared
Found 151 images belonging to 1 classes.
Found 151 images belonging to 1 classes.
1/1 [==============================] - 4s 4s/step


In [0]:
# Save predictions
save_result(results, test_root, output_root, gt_root)